In [ ]:
import numpy as np 
import pandas as pd
import pathlib
from scipy.interpolate import CubicSpline

In [ ]:
##############   1. prepare raw CDS data (Bloomberg)    #################

# Bloomberg

COUNTRY_NUM = 15
OUTPUT_REPO = "EXT_data"

for i in ['1', '3', '5']:
    CDS1Y = pd.read_excel("raw/bbg_cds_data.xlsx", sheet_name=f"CDS{i}Y_values", dtype=str, na_values=['-1'], parse_dates=[0], index_col=0)
    # keep non-NA columns
    CDS_columns = [str(col) for col in CDS1Y.columns if "0" not in str(col)]
    CDS1Y = CDS1Y[CDS_columns[:COUNTRY_NUM]].astype(float)
    CDS1Y.to_csv(f"inputs/{OUTPUT_REPO}/CDS{i}Y.csv")
    # CDS1Y.loc['2003-02-01':].iloc[:85].to_csv(f"inputs/{OUTPUT_REPO}/CDS{i}Y.csv")



In [221]:
##############   1. prepare raw CDS data (Markit)    #################

OUTPUT_REPO = "MKT_data"
raw = pd.read_csv("raw/markit_cds.csv")
tickers = pd.read_csv("raw/markit_tickers.txt", header=None)[0].values


for tenor in ['1Y', '3Y', '5Y']:

    concat_list = []
    # aggregate raw data
    # long to wide
    # daily to monthly
    for j, country in enumerate(tickers):
        temp = raw[(raw['Tenor'] == tenor) & (raw['Ticker'] == country)]
        temp = temp.set_index(pd.to_datetime(temp['Date'], format="%Y%m%d"))
        temp = temp.resample('1M').last()['ParSpread'] * 10000
        temp.name = country
        # save for concatenation
        concat_list.append(temp)

    res = pd.concat(concat_list, axis='columns', join='outer')
    res.to_csv(f"inputs/{OUTPUT_REPO}/raw/CDS{tenor}.csv")
    
    # clean up
    concat_list = []
    for j, country in enumerate(tickers):
        
        # generate rows to have consecutive date values
        country_df = res[country].reset_index().dropna()
        t0, t1 = country_df['Date'].values[0], country_df['Date'].values[-1]
        time_index = pd.date_range(start=t0, end=t1, freq='1M')

        # dataframe from t0 to t1 with consecutive dates
        country_df = pd.DataFrame(index=time_index)
        country_df['Date'] = time_index
        country_df[country] = res[country]

        # computes pct of missing data in consecutive time period
        num_NA = int(country_df[country].isna().sum())
        pct_missing = num_NA / len(time_index)

        if pct_missing >= 0.05: # >= 5%; take longest consecutive subset of CDS data

            # drop na to allow partition of sub df
            country_df = country_df.dropna()

            # assign True if 1 month apart from subsequent date
            # `in_block` gives which data are in consecutive block
            in_block = (country_df['Date'].diff()/np.timedelta64(1, 'M')).round() == 1
            filtered = country_df.loc[in_block]

            # `breaks` indicate the start of each group 
            # `groups` assign integer to each group
            breaks = (filtered['Date'].diff() / np.timedelta64(1, 'M')).round() != 1
            groups = breaks.cumsum()

            temp = sorted([frame for _, frame in filtered.groupby(groups)], key=lambda x: len(x), reverse=True)[0]
            temp = temp.set_index('Date')
            # concat_list.append(temp)
            # break
        else: # do linear interpolate
            temp = country_df.interpolate(method='linear', inplace=False)
            temp = temp.set_index('Date')

        concat_list.append(temp)
            

    res = pd.concat(concat_list, axis='columns', join='outer')
    res.to_csv(f"inputs/{OUTPUT_REPO}/CDS{tenor}.csv")


    # break





In [ ]:



##############   2. construct discount value; boostrap Treasury constant maturity curve       #################
# partial credit to Steven Zheng

PFILE = pd.read_csv("raw/FRB_H15.csv", header=0, skiprows=[1,2,3,4,5], dtype=str, na_values=['ND'], parse_dates=[0])

# rename columns using maturity in weeks
new_cols = ['date', 4, 13, 26, 52, 104, 156, 260, 364, 520, 1040, 1560]
rename_dict = { col: new_cols[i] for i, col in enumerate(PFILE.columns)}
PFILE.rename(columns=rename_dict, inplace=True)
# set index, and set values to numeric, % to unit less
PFILE = PFILE.set_index('date')
PFILE = PFILE.astype({col:np.float for col in PFILE.columns}) / 100
# compute discount value
for col in PFILE.columns:
    PFILE[col] = PFILE[col].apply(func= lambda x: 1. / ( (1 + x) ** (float(col) / 52) ))
PFILE = PFILE.dropna(axis=0)

In [ ]:
# interpolate to fill weekly maturities using cubic spline
tenor = PFILE.columns.astype(int)
inter_PFILE_list = list()

for i, row in PFILE.iterrows():
    cs = CubicSpline(x=tenor, y=row.values)
    inter_PFILE_list.append([float(cs(t)) for t in range(1,261)])

inter_PFILE = pd.DataFrame(index=PFILE.index, columns=range(1,261), data=inter_PFILE_list)

In [ ]:
# PFILE
inter_PFILE.to_csv("inputs/Test_data/PFILE.csv")





